In [2]:
import pandas as pd
import numpy as np
from src.scrap import parse_machine_kg_one_page, get_all_car_models_with_selenium

# 1.Parsing 

In [2]:
url = "https://www.mashina.kg/search/all/?region=all"

models = get_all_car_models_with_selenium(url)

columns = [
        "mark", "model",
        'color',
        'year', 'v_engine', 'gearbox',
        'body_type', 'fuel_type',
        'sw_location', 'mileage',
        "location",
        "price_in_usd"
    ]

In [11]:
last_page = 1755  # 27.09.2024

data = np.concatenate(
    [
        await parse_machine_kg_one_page(
            url=url,
            models=models,
            params={"page": page}
        )
        for page in range(1, last_page + 1)
    ]
)


page 1 parsed!
page 2 parsed!
page 3 parsed!
page 4 parsed!
page 5 parsed!
page 6 parsed!
page 7 parsed!
page 8 parsed!
page 9 parsed!
page 10 parsed!
page 11 parsed!
page 12 parsed!
page 13 parsed!
page 14 parsed!
page 15 parsed!
page 16 parsed!
page 17 parsed!
page 18 parsed!
page 19 parsed!
page 20 parsed!
page 21 parsed!
page 22 parsed!
page 23 parsed!
page 24 parsed!
page 25 parsed!
page 26 parsed!
page 27 parsed!
page 28 parsed!
page 29 parsed!
page 30 parsed!
page 31 parsed!
page 32 parsed!
page 33 parsed!
page 34 parsed!
page 35 parsed!
page 36 parsed!
page 37 parsed!
page 38 parsed!
page 39 parsed!
page 40 parsed!
page 41 parsed!
page 42 parsed!
page 43 parsed!
page 44 parsed!
page 45 parsed!
page 46 parsed!
page 47 parsed!
page 48 parsed!
page 49 parsed!
page 50 parsed!
page 51 parsed!
page 52 parsed!
page 53 parsed!
page 54 parsed!
page 55 parsed!
page 56 parsed!
page 57 parsed!
page 58 parsed!
page 59 parsed!
page 60 parsed!
page 61 parsed!
page 62 parsed!
page 63 parsed!
p

In [12]:
df = pd.DataFrame(
    data=data,
    columns=columns
)

In [13]:
df.head()

,mark,model,color,year,v_engine,gearbox,body_type,fuel_type,sw_location,mileage,location,price_in_usd
0,BMW,X7,черный,2021г.,3л.,автомат,внедорожник5дв.,бензин,руль слева,119 000 км,Бишкек,$72000
1,Mercedes-Benz,A-Класс,серый,2022г.,2л.,робот,седан,бензин,руль слева,18 000 км,Бишкек,$28500
2,Toyota,Camry,синий,2012г.,2.5л.,вариатор,седан,гибрид,руль слева,255 885 км,Бишкек,$14400
3,BMW,M3,серый,2021г.,3л.,автомат,седан,бензин,руль слева,32 000 км,Бишкек,$85000
4,Toyota,Land Cruiser Prado,белый,2022г.,2.7л.,автомат,внедорожник5дв.,бензин,руль слева,70 000 км,Бишкек,$58000


In [14]:
df.to_csv("../data/raw.csv", index=False)

# Data Preprocessing

In [3]:
df = pd.read_csv("../data/raw.csv")

In [4]:
X, y = df.drop("price_in_usd", axis=1), df["price_in_usd"]
y = y.str.replace("$", "", regex=False).astype(np.int32)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [6]:
from src.preprocessing import PreparingDataService

processing_service = PreparingDataService()
X_train = processing_service.fit_transform(X_train)
X_test = processing_service.transform(X_test)

In [7]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
X_tr = X_train.copy()
X_tr = std.fit_transform(X_tr)


X_te = X_test.copy()
X_te = std.transform(X_te)

# Model Evaluation

In [8]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

lg = Ridge(alpha=1)
lg.fit(X_tr, y_train)
print(f'Ridge reg: {lg.score(X_te, y_test)}')

rn = RandomForestRegressor(min_samples_split=50)
rn.fit(X_tr, y_train)
print(f'Random Forest: {rn.score(X_te, y_test)}')

Ridge reg: 0.42535857045903624
Random Forest: 0.8356322809828413


In [13]:
from sklearn.model_selection import cross_val_score

f"Random Forest cross val score: {np.mean(cross_val_score(rn, X_tr, y_train))}"

'Random Forest cross val score: 0.8360427812161539'

In [15]:
pd.DataFrame(data=rn.feature_importances_, index=X_train.columns)

,0
mark,0.165920
model,0.031096
year,0.322548
v_engine,0.388701
sw_location,0.000041
mileage,0.038888
PCA0,0.003104
PCA1,0.004126
PCA2,0.004055
PCA3,0.003345
